In [1]:
from model_v2 import build_lm_classifier_inference, LSTM_SAVED_STATE
from utils import get_batch_classifier_inference, clean_text
import json
import tensorflow as tf
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
%matplotlib inline

In [2]:
with open('intent_102/checkpoints/model_configs.json', 'r') as inp:
    lm_params = json.load(inp)
with open('intent_102/checkpoints/classifier_configs.json', 'r') as inp:
    cls_params = json.load(inp)
# Load in CPU
language_model, classifier = build_lm_classifier_inference(lm_params, cls_params, is_cpu=True)

In [3]:
config = tf.ConfigProto()
config.gpu_options.allow_growth = True
session = tf.Session(config=config)
saver = tf.train.Saver([x for x in tf.global_variables() if x not in tf.get_collection(LSTM_SAVED_STATE)])
saver.restore(session, 'intent_102/checkpoints/classifier_interest_cpu/model.cpkt-150')

INFO:tensorflow:Restoring parameters from intent_102/checkpoints/classifier_interest_cpu/model.cpkt-150


In [4]:
with open('IntentDetection/word2idx.json', 'r') as inp:
    word2idx = json.load(inp)
with open('IntentDetection/char2idx.json', 'r') as inp:
    char2idx = json.load(inp)
word2char = {w: [char2idx[c] for c in w] for w in word2idx}

In [5]:
def predict(texts, bsz=32):
    texts = [clean_text(x.strip()) for x in texts]
    texts = np.array([[word2char.get(w, word2char['<UNK>']) for w in sent.split()] for sent in texts])
    results = []
    for chars, lens, char_lens in get_batch_classifier_inference(texts, bsz):
        probs = session.run(classifier.probs, feed_dict={
            language_model.inputs: chars, language_model.seq_lens: lens,
            language_model.char_lens: char_lens, language_model.bptt: 20
        })
        results.append(probs)
    return np.concatenate(results, axis=0)

In [6]:
data = pd.read_csv('IntentDetection/total_purchase_text - totalpurchase_text.csv', names=['label', 'mention', '1', '2'], skiprows=1)[['label', 'mention']]

In [7]:
data.dropna(inplace=True)
data['interest'] = data['label'].map(lambda x: int('I' in x))

In [ ]:
data['pred'] = np.argmax(predict(data['mention']), axis=1)